<a href="https://colab.research.google.com/github/SHUBHAM12012005/Vaccine-Prediction/blob/main/final_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np

train_features = pd.read_csv('/content/drive/My Drive/hackathon1/training_set_features.csv')
train_labels = pd.read_csv('/content/drive/My Drive/hackathon1/training_set_labels.csv')
test_features = pd.read_csv('/content/drive/My Drive/hackathon1/test_set_features.csv')

df = train_features.merge(train_labels, on='respondent_id')
y = df[['xyz_vaccine', 'seasonal_vaccine']]
X = df.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'])

numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = MultiOutputClassifier(GradientBoostingClassifier(random_state=42))

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred_proba = pipeline.predict_proba(X_val)

y_pred_proba_df = pd.DataFrame(np.hstack([y_pred_proba[0][:, 1].reshape(-1, 1), y_pred_proba[1][:, 1].reshape(-1, 1)]),
                               columns=['xyz_vaccine', 'seasonal_vaccine'])

roc_auc_xyz = roc_auc_score(y_val['xyz_vaccine'], y_pred_proba_df['xyz_vaccine'])
roc_auc_seasonal = roc_auc_score(y_val['seasonal_vaccine'], y_pred_proba_df['seasonal_vaccine'])
mean_roc_auc = (roc_auc_xyz + roc_auc_seasonal) / 2

print(f"ROC AUC for xyz_vaccine: {roc_auc_xyz}")
print(f"ROC AUC for seasonal_vaccine: {roc_auc_seasonal}")
print(f"Mean ROC AUC: {mean_roc_auc}")

test_pred_proba = pipeline.predict_proba(test_features.drop(columns=['respondent_id']))

test_pred_proba_df = pd.DataFrame(np.hstack([test_pred_proba[0][:, 1].reshape(-1, 1), test_pred_proba[1][:, 1].reshape(-1, 1)]),
                                  columns=['xyz_vaccine', 'seasonal_vaccine'])

submission = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': test_pred_proba_df['xyz_vaccine'],
    'seasonal_vaccine': test_pred_proba_df['seasonal_vaccine']
})

submission.to_csv('/content/drive/My Drive/hackathon1/submission.csv', index=False)
print("Submission file created successfully.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ROC AUC for xyz_vaccine: 0.838986692047164
ROC AUC for seasonal_vaccine: 0.8608546254424846
Mean ROC AUC: 0.8499206587448243
Submission file created successfully.
